In [8]:
import cv2
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier

In [14]:
X, y = make_classification(n_samples=1000, n_features=4,
                            n_informative=2, n_redundant=0,
                            random_state=0, shuffle=False)
base_classifier=DecisionTreeClassifier(max_depth=2)
clf = AdaBoostClassifier(base_classifier, 2048, random_state=0)
print('X',y.shape)
clf.fit(X, y)
clf.predict([[0, 0, 0, 0]])
#clf.score(X, y)

X (1000,)


1

In [15]:
#ACF:
#1237 osób
#4280 tła - zupełnie bez palety

Viano
7124/6700 zdjęć
9047 palet
37 scen
181 obrazów per scena

Można wziąć 20 procent zróżnicowanych scen, czyli kilka i na nich nauczyć. 
Negatywnych wziąć losowo 2x więcej niż pozytywnych.

Tool do zaznaczania może otwierać po kolei obrazy z danego folderu 
i przechwytywać podwójne kliknięcia oznaczające lewy górny i prawy dolny róg palety.
Obrazy z aparatu można zmniejszyć do wielkości 1080/720 jak obrazy z kamery.
Do tego można wyświetlać linie poziomą i pionową pomocniczą. 
Ustalić, że np. prawy klawisz kończy zaznaczanie.
Zapisywać wszystko w pliku, gdzie linię rozpoczyna nazwa pliku, a potem są pary punktów.